# A Study of the Effect of YouTube Tech Channels to the Units Sold of Newly Released Devices

In [2]:
# Libraries used

import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent
import time, os
import numpy as np
import sys
from tqdm import tqdm
from api import *
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors


In [3]:
# pip install --upgrade google-auth-oauthlib google-auth-httplib2

In [4]:
# pip install --upgrade google-api-python-client

## Scrapping

### In this section, we will scrap Wikipedia to get the financial information we want for each device.

In [5]:
url = "https://en.wikipedia.org/wiki/List_of_best-selling_mobile_phones#Top-selling_mobile_phones"
wikipage = requests.get(url)
soup = BeautifulSoup(wikipage.text, "html5lib")

print(wikipage.status_code)

200


In [6]:
columns = soup.find('table', class_='wikitable').find('tr') 

In [7]:
column_names = columns.text
column_names = column_names.replace('\n','  ').strip().split('  ')
column_names = [value for value in column_names if value != '']
column_names.append('x')

In [8]:
data = soup.find('table', class_='wikitable').find('tbody')

In [9]:
devices_list = []
for row in data.find_all('tr')[1:]:
    devices_list.append(row.text.strip().split('\n\n'))

In [10]:
units_sold_df = pd.DataFrame(devices_list, columns=column_names)
units_sold_df

,Manufacturer,Model,Form factor,Smartphone?,Year,Graph (million units),x
0,Nokia,1100,Bar,No,2003,250[3][1][4],250
1,Nokia,1110,Bar,No,2005,250[3][1][5],250
2,Apple,iPhone 6 and iPhone 6 Plus,Touchscreen,Yes,2014,222[2][3],222
3,Nokia,105 (2013) and 105 (2015),Bar,No,2013,200[6],200
4,Nokia,3210,Bar,No,1999,160[1][5][7],160
...,...,...,...,...,...,...,...
110,LG,G2,Touchscreen,Yes,2013,3[62][63],3
111,LeTV,LeEco Le 1s,Touchscreen,Yes,2016,3[64][65],3
112,Google,Pixel and Pixel XL,Touchscreen,Yes,2016,2.1[66],2.1
113,Palm,Centro,Keyboard bar,Yes,2007,2[67],2


In [11]:
units_sold_df.drop(columns=['Graph (million units)', 'Form factor'],inplace=True)
units_sold_df.head()

,Manufacturer,Model,Smartphone?,Year,x
0,Nokia,1100,No,2003,250
1,Nokia,1110,No,2005,250
2,Apple,iPhone 6 and iPhone 6 Plus,Yes,2014,222
3,Nokia,105 (2013) and 105 (2015),No,2013,200
4,Nokia,3210,No,1999,160


In [12]:
units_sold_df.rename(columns={'Manufacturer':'company_name', 
                              'Model':'device', 
                              'x':'units', 
                              'Smartphone?':'smartphone?', 
                              'Year': 'year'},inplace=True)

In [13]:
units_sold_df = units_sold_df[units_sold_df['smartphone?'] == 'Yes'].reset_index(drop=True).drop(columns=['smartphone?'])

In [14]:
units_sold_df['year'] = units_sold_df['year'].astype('int32')
units_sold_df['units'] = units_sold_df['units'].astype('float32') * 1000000

In [15]:
units_sold_df['company_name'] = units_sold_df.company_name.str.upper()
units_sold_df['device'] = units_sold_df.device.str.upper()

In [16]:
units_sold_df.head()

,company_name,device,year,units
0,APPLE,IPHONE 6 AND IPHONE 6 PLUS,2014,2.220000e+08
1,NOKIA,6600,2003,1.500000e+08
2,NOKIA,5230,2009,1.500000e+08
3,APPLE,IPHONE 11,2019,1.021000e+08
4,APPLE,IPHONE 12 AND IPHONE 12 MINI,2020,1.000000e+08


We notice that in some cases, there are two devices in the 'device' column. Next, we will clean it.

In [17]:
cn = []                 # company_name
d = []                  # device
y = []                  # year
u = []                  # year

for a_string in units_sold_df.device:
    for a_list in a_string.split('AND'):
        
        cn.append(units_sold_df[units_sold_df['device'] == a_string].company_name.values[0])
        d.append(a_list.split(',')[0].strip().upper())
        y.append(units_sold_df[units_sold_df['device'] == a_string].year.values[0])
        u.append(units_sold_df[units_sold_df['device'] == a_string].units.values[0])

units_sold_df_mod = pd.DataFrame({'company_name': cn,
                                  'device': d,
                                  'year': y,
                                  'units': u})

In [18]:
units_sold_df_mod.head()

,company_name,device,year,units
0,APPLE,IPHONE 6,2014,2.220000e+08
1,APPLE,IPHONE 6 PLUS,2014,2.220000e+08
2,NOKIA,6600,2003,1.500000e+08
3,NOKIA,5230,2009,1.500000e+08
4,APPLE,IPHONE 11,2019,1.021000e+08


### In this section, we will scrap (https://www.gsmarena.com/makers.php3) to get all companies names, devices and number of devices.

In [19]:
the_link = 'https://www.gsmarena.com/makers.php3'
ua = UserAgent()
def get_soup(the_link):
    source_code = requests.get(the_link, headers = {'User-agent': ua.random})
    return BeautifulSoup(source_code.text, 'html5lib')

In [20]:
soup = get_soup(the_link)
list_of_companies = soup.find('div', {'class': 'st-text'})

company_name = []
links = []
number_of_devices = []

for anchor in list_of_companies.findAll('a'):
    company_name.append(anchor.text.strip())
    links.append('https://www.gsmarena.com/'+anchor.attrs['href'])
    number_of_devices.append(anchor.find('span').text)

We notice that companies names and the number of devices in company_name list are compined into a single string as shown below.

In [21]:
print(company_name[:2])
print(number_of_devices[:2])

['Acer100 devices', 'alcatel405 devices']
['100 devices', '405 devices']


We will now clean the data

In [22]:
for i in range(len(company_name)):
    company_name[i] = company_name[i].replace(number_of_devices[i],'')

In [23]:
# Now data is cleaned
print(company_name[:2])
print(number_of_devices[:2])

['Acer', 'alcatel']
['100 devices', '405 devices']


In [24]:
df1 = pd.DataFrame()
df1['company_name'] = company_name
df1['link'] = links
df1['number_of_devices'] = [int(i.replace('devices','').strip()) for i in number_of_devices]

In [25]:
df1['company_name'] = df1.company_name.str.upper()

In [26]:
df2 = pd.merge(df1, units_sold_df_mod, on='company_name')[['company_name', 'link']].drop_duplicates().reset_index(drop=True)

In [31]:
df2

,company_name,link
0,APPLE,https://www.gsmarena.com/apple-phones-48.php
1,GOOGLE,https://www.gsmarena.com/google-phones-107.php
2,HTC,https://www.gsmarena.com/htc-phones-45.php
3,HUAWEI,https://www.gsmarena.com/huawei-phones-58.php
4,LG,https://www.gsmarena.com/lg-phones-20.php
5,MOTOROLA,https://www.gsmarena.com/motorola-phones-4.php
6,NOKIA,https://www.gsmarena.com/nokia-phones-1.php
7,OPPO,https://www.gsmarena.com/oppo-phones-82.php
8,PALM,https://www.gsmarena.com/palm-phones-27.php
9,SAMSUNG,https://www.gsmarena.com/samsung-phones-9.php


Now we wand to scrap on the companies that we have its devices number of units sold.

In [38]:
devices = []
companies_names = []
for a_link in tqdm(df2.link.values):   #for a_link in ['https://www.gsmarena.com/acer-phones-59.php']:
    soup = get_soup(a_link)
    cn = df1[df1['link'] == a_link]['company_name'].values[0]
    
    # We'll now generate a list with all the pages containing a company's device 
    all_pages_links = [a_link]
    list_of_pages = soup.find('div', {'class': 'nav-pages'})
    try:
        for i in list_of_pages.find_all('a'):
            all_pages_links.append('https://www.gsmarena.com/'+i.get('href'))
    except:
        pass
    
    # We'll get all the devices names 
    for page_link in all_pages_links:
        soup = get_soup(page_link)
        list_of_devices = soup.find('div', {'class': 'makers'})
        time.sleep(1+np.random.randint(0,1))
        try:
            for name in list_of_devices.find_all('span'):
                time.sleep(1+np.random.randint(0,1))
                devices.append(name.text)
                companies_names.append(cn)
        except:
            devices.append(list_of_pages.find('span').text)
    time.sleep(1+np.random.randint(0,1))

100%|███████████████████████████████████████████████████████████████████████████████| 11/11 [1:13:23<00:00, 400.27s/it]


In [29]:
# Now we will create a dataframe that contains the companies names and the names of the devices
CN_D = pd.DataFrame({'company_name':companies_names, 'device':devices})
CN_D.to_csv('CN_D.csv', index = False)

Since scrapping takes alot of time to run, we'll use the saved data in the csv file.

In [32]:
CN_D = pd.read_csv('CN_D.csv')
CN_D

,company_name,device
0,APPLE,iPhone 13 Pro Max
1,APPLE,iPhone 13 Pro
2,APPLE,iPhone 13
3,APPLE,iPhone 13 mini
4,APPLE,iPad mini (2021)
...,...,...
4207,XIAOMI,Mi Max 4 Pro
4208,XIAOMI,Mi Max 4
4209,XIAOMI,Mi 6c
4210,XIAOMI,Redmi Pro 2


In [33]:
# Now we will join the two dataframes
df_devices = pd.merge(df1, CN_D, on='company_name')

In [34]:
df_devices['device'] = df_devices.device.str.upper()

In [35]:
df_devices

,company_name,link,number_of_devices,device
0,APPLE,https://www.gsmarena.com/apple-phones-48.php,98,IPHONE 13 PRO MAX
1,APPLE,https://www.gsmarena.com/apple-phones-48.php,98,IPHONE 13 PRO
2,APPLE,https://www.gsmarena.com/apple-phones-48.php,98,IPHONE 13
3,APPLE,https://www.gsmarena.com/apple-phones-48.php,98,IPHONE 13 MINI
4,APPLE,https://www.gsmarena.com/apple-phones-48.php,98,IPAD MINI (2021)
...,...,...,...,...
4207,XIAOMI,https://www.gsmarena.com/xiaomi-phones-80.php,229,MI MAX 4 PRO
4208,XIAOMI,https://www.gsmarena.com/xiaomi-phones-80.php,229,MI MAX 4
4209,XIAOMI,https://www.gsmarena.com/xiaomi-phones-80.php,229,MI 6C
4210,XIAOMI,https://www.gsmarena.com/xiaomi-phones-80.php,229,REDMI PRO 2


### In the next section, we will scrap YouTube to get the length of the video, number of like and dislikes, the channels names, and the devices the reviews were about.

In [148]:
videoIds=[]
scopes = ["https://www.googleapis.com/auth/youtube.readonly"]
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

In [82]:
# SETUP
api_service_name = "youtube"
api_version = "v3"
token = api_token
youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey=token)

In [80]:
# Three of the most popular Tech channels were selected
channels_names = ['marquesbrownlee','unboxtherapy','Mrwhosetheboss']
ranges = [] # Number of videos devided by 50

In [89]:
def get_upload_list(a_channel_name):
    uploads_lists = []
    # Get Channel details (Find the uploads playlist id from it)
    channelReq = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        forUsername=a_channel_name
    )
    channelRes = channelReq.execute()
    uploads_lists.append(channelRes['items'][0]['contentDetails']['relatedPlaylists']['uploads'])
    
    
    return uploads_lists

In [92]:
# # channelReq = youtube.channels().list(
#     part="snippet,contentDetails,statistics",
#     forUsername='marquesbrownlee'
# )
# channelRes = channelReq.execute()

In [104]:
# channelRes['items'][0]['contentDetails']['relatedPlaylists']['uploads']

'UUBJycsmduvYEL83R_U4JriQ'

In [105]:
# get_upload_list(channels_names)

In [151]:
def get_vids_ids(channels_names, uploads_list, ranges):
    videos_Ids=[]
    channels_names_modified = []
    
    for i in range(len(channels_names)):
        # ---------------------------------------------------------------------------------
        # Get the uploads of a channel
        uploadsReq = youtube.playlistItems().list(
            part="snippet,contentDetails",
            maxResults=50,
            playlistId=uploads_list[i],
        )
        uploadsRes = uploadsReq.execute()
        for video in range(len(uploadsRes['items'])):
            videos_Ids.append(uploadsRes['items'][video]['contentDetails']['videoId'])
            channels_names_modified.append(channels_names[i])
        # ---------------------------------------------------------------------------------    
        NEXT_PAGE_TOKEN = uploadsRes['nextPageToken']
        for page in range(ranges[i]): ####################### What is it used for? or how to get it? Can we generalize? 
            # Get the uploads of a channel
            nextUploadsReq = youtube.playlistItems().list(
                part="snippet,contentDetails",
                maxResults=50,
                playlistId=uploads_list[i],
                pageToken=NEXT_PAGE_TOKEN
            )
            nextUploadsRes = nextUploadsReq.execute()
            try:
                NEXT_PAGE_TOKEN = nextUploadsRes['nextPageToken']
                for video in range(len(nextUploadsRes['items'])):
                    videos_Ids.append(nextUploadsRes['items'][video]['contentDetails']['videoId'])
                    channels_names_modified.append(channels_names[i])
            except:
                print('No next page on:   '+ channels_names[i])
    return videos_Ids, channels_names_modified

In [152]:
def get_vid_details(videoIds, channels_names_modified):
    titles = []
    tags = []
    likeCount = []
    viewCount = []
    dislikeCount = []
    commentCount = []
    # ---------------------------------------------------------------------------------
    # Get channel's stats videos details
    fifties = 0
    for iter in range(len(videoIds)):
        vidReq = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=videoIds[iter]
        )
        vidRes = vidReq.execute()
        try:
            viewCount.append(vidRes['items'][0]['statistics']['viewCount'])
        except:
            print('video with no viewCount') 
            viewCount.append(None)
        try:
            likeCount.append(vidRes['items'][0]['statistics']['likeCount'])
        except:
            print('video with no likeCount') 
            likeCount.append(None)
        try:
            dislikeCount.append(vidRes['items'][0]['statistics']['dislikeCount'])
        except:
            print('video with no dislikeCount') 
            dislikeCount.append(None)
        try:
            commentCount.append(vidRes['items'][0]['statistics']['commentCount'])
        except:
            print('video with no commentCount') 
            commentCount.append(None)
        try:
            titles.append(vidRes['items'][0]['snippet']['title'])
        except:
            print('video with no titles') 
            titles.append(None)
        try:
            tags.append(vidRes['items'][0]['snippet']['tags'])
        except:
            print('video with no tags') 
            tags.append(None)
            
    df = pd.DataFrame({'channel_name': channels_names_modified, 
                         'titles': titles, 
                         'tags': tags, 
                         'likes_number': likeCount, 
                         'views count': viewCount,
                         'dislikes_number': dislikeCount, 
                         'commens_count': commentCount})
    return df ################################################### GET THE DURATION !!

In [ ]:
yt_df.to_csv('youtube_scrapping_data.csv')

In [38]:
yt_df = pd.read_csv('youtube_data.csv', index_col=0).reset_index(drop=True)

In [40]:
# H M S
# H M
# H   S
# H 
#   M S
#   M
#     S

for i in range(yt_df.shape[0]):
    x = yt_df.video_duration.str.strip('PT').iloc[i]
    if 'H' in x and 'M' in x and 'S' in x:
        h = int(x.split('H')[0])
        m = int(x.split('H')[1].split('M')[0])
        s = int(x.split('H')[1].split('M')[1].strip('S'))
        yt_df['video_duration'].iloc[i] = (h * 60 * 60) + (m * 60) + s
        
    elif 'H' in x and 'M' in x and 'S' not in x:
        h = int(x.split('H')[0])
        m = int(x.split('H')[1].strip('M'))
        yt_df['video_duration'].iloc[i] = (h * 60 * 60) + (m * 60)
        
    elif 'H' in x and 'M' not in x and 'S' in x:
        h = int(x.split('H')[0])
        s = int(x.split('H')[1].strip('S'))
        yt_df['video_duration'].iloc[i] = (h * 60 * 60) + s
                
    elif 'H' in x and 'M' not in x and 'S' not in x:
        h = int(x.strip('H'))
        yt_df['video_duration'].iloc[i] = (h * 60 * 60)
        
    elif not 'H' in x and 'M' in x and 'S' in x:
        m = int(x.split('M')[0])
        s = int(x.split('M')[1].strip('S'))
        yt_df['video_duration'].iloc[i] = (m * 60) + s
        
    elif not 'H' in x and 'M' in x and 'S' not in x:
        m = int(x.strip('M'))
        yt_df['video_duration'].iloc[i] = (m * 60)
        
    elif not 'H' in x and 'M' not in x and 'S' in x:
        s = int(x.strip('S'))
        yt_df['video_duration'].iloc[i] = s
    

C:\Users\Windows10\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [46]:
yt_df.tags.iloc[0]

"['surface laptop', 'surface studio', 'surface laptop studio', 'mkbhd', 'MKBHD', 'Microsoft Surface', 'Microsoft Surface Laptop Studio', 'Windows 11']"

In [41]:
yt_df['channel_name'] = yt_df.channel_name.str.upper()

In [42]:
yt_df.dropna(inplace = True)
yt_df.reset_index(drop=True, inplace = True)

In [47]:
channel_name = []
titles = []
tags = []
likes_number = []
views_count = []
dislikes_number = []
commens_count = []
video_duration = []
for ii in range(yt_df.shape[0]): 
    for i in yt_df.tags.iloc[ii].strip("'[]'").split(','):
        channel_name.append(yt_df.channel_name.iloc[ii])
        tags.append(i.strip().strip("''").strip())
        titles.append(yt_df.titles.iloc[ii])
        likes_number.append(yt_df.likes_number.iloc[ii])
        dislikes_number.append(yt_df.dislikes_number.iloc[ii])
        commens_count.append(yt_df.commens_count.iloc[ii])
        video_duration.append(yt_df.video_duration.iloc[ii])
        views_count.append(yt_df['views count'].iloc[ii])

In [48]:
yt_df_modified = pd.DataFrame({'channel_name': channel_name,
                               'titles': titles,
                               'tags': tags,
                               'likes_number': likes_number,
                               'views_count': views_count,
                               'dislikes_number': dislikes_number,
                               'commens_count': commens_count,
                               'video_duration': video_duration})

In [49]:
yt_df_modified['tags'] = yt_df_modified.tags.str.upper()

In [50]:
yt_df_modified

,channel_name,titles,tags,likes_number,views_count,dislikes_number,commens_count,video_duration
0,MKBHD,Surface Laptop Studio Impressions: Windows 11 ...,SURFACE LAPTOP,80208.0,2184860.0,1472.0,5720.0,687
1,MKBHD,Surface Laptop Studio Impressions: Windows 11 ...,SURFACE STUDIO,80208.0,2184860.0,1472.0,5720.0,687
2,MKBHD,Surface Laptop Studio Impressions: Windows 11 ...,SURFACE LAPTOP STUDIO,80208.0,2184860.0,1472.0,5720.0,687
3,MKBHD,Surface Laptop Studio Impressions: Windows 11 ...,MKBHD,80208.0,2184860.0,1472.0,5720.0,687
4,MKBHD,Surface Laptop Studio Impressions: Windows 11 ...,MKBHD,80208.0,2184860.0,1472.0,5720.0,687
...,...,...,...,...,...,...,...,...
127117,UNBOX_THERAPY,Lacie 2big Quadra Unboxing & Overview in HD!,HOT,170.0,10440.0,5.0,17.0,283
127118,UNBOX_THERAPY,Lacie 2big Quadra Unboxing & Overview in HD!,SWAP,170.0,10440.0,5.0,17.0,283
127119,UNBOX_THERAPY,Lacie 2big Quadra Unboxing & Overview in HD!,7200RPM,170.0,10440.0,5.0,17.0,283
127120,UNBOX_THERAPY,Lacie 2big Quadra Unboxing & Overview in HD!,32MB,170.0,10440.0,5.0,17.0,283


### Final DataFrame preparation.

In [52]:
df_devices.head(1)

,company_name,link,number_of_devices,device
0,APPLE,https://www.gsmarena.com/apple-phones-48.php,98,IPHONE 13 PRO MAX


In [53]:
yt_df_modified.head(1)

,channel_name,titles,tags,likes_number,views_count,dislikes_number,commens_count,video_duration
0,MKBHD,Surface Laptop Studio Impressions: Windows 11 ...,SURFACE LAPTOP,80208.0,2184860.0,1472.0,5720.0,687


In [54]:
units_sold_df_mod.head(1)

,company_name,device,year,units
0,APPLE,IPHONE 6,2014,222000000.0


In [60]:
pre_final_df = pd.merge(df_devices, 
                        units_sold_df_mod, 
                        on='device').drop(['number_of_devices', 'link', 'company_name_y'], axis=1)

In [68]:
yt_df_modified.head(1)

,channel_name,titles,tags,likes_number,views_count,dislikes_number,commens_count,video_duration
0,MKBHD,Surface Laptop Studio Impressions: Windows 11 ...,SURFACE LAPTOP,80208.0,2184860.0,1472.0,5720.0,687


In [67]:
df = pd.merge(pre_final_df, 
              yt_df_modified, 
              left_on = 'device', 
              right_on = 'tags').drop(['tags', 'titles'], axis=1)

In [70]:
df

(4898, 10)

In [75]:
df.drop_duplicates(inplace = True, ignore_index = True)

In [76]:
df

,company_name_x,device,year,units,channel_name,likes_number,views_count,dislikes_number,commens_count,video_duration
0,APPLE,IPHONE 12 PRO MAX,2020,2.900000e+07,MKBHD,289750.0,6380863.0,6569.0,19865.0,1505
1,APPLE,IPHONE 12 PRO MAX,2020,2.900000e+07,MKBHD,170900.0,5512865.0,3293.0,12763.0,609
2,APPLE,IPHONE 12 PRO MAX,2020,2.900000e+07,MKBHD,230897.0,6736834.0,6498.0,27074.0,1105
3,APPLE,IPHONE 12 PRO MAX,2020,2.900000e+07,MRWHOSETHEBOSS,226495.0,3954624.0,6898.0,23637.0,968
4,APPLE,IPHONE 12 PRO MAX,2020,2.900000e+07,MRWHOSETHEBOSS,155226.0,4382765.0,2691.0,13550.0,593
...,...,...,...,...,...,...,...,...,...,...
4857,SAMSUNG,GALAXY S21,2021,1.350000e+07,MRWHOSETHEBOSS,266979.0,7767749.0,4771.0,17198.0,1093
4858,XIAOMI,REDMI 8,2019,6.800000e+06,MRWHOSETHEBOSS,68267.0,3046416.0,2970.0,2037.0,436
4859,XIAOMI,REDMI NOTE 7 PRO,2019,2.000000e+07,UNBOX_THERAPY,36921.0,1248998.0,1057.0,5495.0,393
4860,XIAOMI,REDMI NOTE 7,2019,2.000000e+07,MRWHOSETHEBOSS,68267.0,3046416.0,2970.0,2037.0,436


In [77]:
df.to_csv('final_df.csv')

## EDA

## Regression and Experiments

In [ ]:
#  
# 
# 
# 
# find a way to join YT (tags)
# EDA
# Regression
# Final Analysis and conclusion 
# Presentation